<a href="https://colab.research.google.com/github/miguelangel18241/Probabilities/blob/main/ASSESSMENT_3_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torchaudio


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

Women's audio

In [3]:
from google.colab import files
tat_q_1 = files.upload()
tat_q_2 = files.upload()
tat_q_3 = files.upload()
tat_q_4 = files.upload()

Saving Tat_question_1.mp3 to Tat_question_1 (1).mp3


Saving Tat_question_2.mp3 to Tat_question_2.mp3


Saving Tat_question_3.mp3 to Tat_question_3.mp3


Saving Tat_question_4.mp3 to Tat_question_4.mp3


Men's audio

In [4]:
from google.colab import files
mig_q_1 = files.upload()
mig_q_2 = files.upload()
mig_q_3 = files.upload()
mig_q_4 = files.upload()

Saving Mig_question_1.mp3 to Mig_question_1.mp3


Saving Mig_question_2.mp3 to Mig_question_2.mp3


Saving Mig_question_3.mp3 to Mig_question_3.mp3


Saving Mig_question_4.mp3 to Mig_question_4.mp3


In [12]:
from google.colab import files

# Upload files recorded by Miguel (man)
mig_files = files.upload()
print("Miguel's files:", list(mig_files.keys()))

# Upload files recorded by Tatiana (girl)
tat_files = files.upload()
print("Tatiana's files:", list(tat_files.keys()))


Saving Mig_question_1.mp3 to Mig_question_1 (1).mp3
Miguel's files: ['Mig_question_1 (1).mp3']


Saving Tat_question_1.mp3 to Tat_question_1 (2).mp3
Tatiana's files: ['Tat_question_1 (2).mp3']


In [23]:
import torch
import torchaudio
import torch.nn as nn
import torchaudio.transforms as transforms
from google.colab import files

# Example using torchaudio with a simple CNN architecture
class GenderCNN(nn.Module):
    def __init__(self):
        super(GenderCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 64 * 64, 128)  # Adjust based on your input size
        self.fc2 = nn.Linear(128, 2)  # Output for binary classification (Male/Female)

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = x.view(-1, 64 * 64 * 64)  # Adjust this to match the flattened size
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Load pre-trained model or train your own on a gender-labeled dataset
model = GenderCNN()

# Function to load and process audio files
def process_audio(file_name):
    waveform, sample_rate = torchaudio.load(file_name)
    mel_spectrogram = transforms.MelSpectrogram()(waveform)

    # Reshape the tensor to match the CNN input format [batch_size, channels, height, width]
    input_tensor = mel_spectrogram.unsqueeze(0)  # Add batch dimension
    input_tensor = input_tensor.mean(dim=1, keepdim=True)  # Convert to single channel if needed

    output = model(input_tensor)
    gender = torch.argmax(output, dim=1).item()
    return "Male" if gender == 0 else "Female"



In [28]:
import torch
import torchaudio
import torch.nn as nn
import torchaudio.transforms as transforms
from google.colab import files

# Example using torchaudio with a simple CNN architecture
class GenderCNN(nn.Module):
    def __init__(self):
        super(GenderCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Adjusted to match the size after conv layers
        self.fc1 = nn.Linear(64 * 32 * 286, 128)
        self.fc2 = nn.Linear(128, 2)  # Output for binary classification (Male/Female)

    def convs(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(x.size(0), -1)  # Flatten before passing to fully connected layers
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Load pre-trained model or train your own on a gender-labeled dataset
model = GenderCNN()

# Function to load and process audio files
def process_audio(file_name):
    waveform, sample_rate = torchaudio.load(file_name)
    mel_spectrogram = transforms.MelSpectrogram()(waveform)

    # Reshape the tensor to match the CNN input format [batch_size, channels, height, width]
    input_tensor = mel_spectrogram.unsqueeze(0)  # Add batch dimension
    input_tensor = input_tensor.mean(dim=1, keepdim=True)  # Convert to single channel if needed

    output = model(input_tensor)
    gender = torch.argmax(output, dim=1).item()
    return "Male" if gender == 0 else "Female"

# Test with one of Miguel's recordings
first_file = list(mig_files.keys())[0]
print(f"Processing Miguel's file: {first_file}")
print("Predicted Gender:", process_audio(first_file))


Processing Miguel's file: Mig_question_1 (1).mp3
Predicted Gender: Male


In [65]:
import zipfile
import os
# Upload and extract the zip file
from google.colab import files
uploaded = files.upload()

zip_file = 'audios.zip'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('audios')

# Process each audio file in the extracted directory
audio_directory = 'audios'

Saving audios.zip to audios (3).zip


In [67]:
new_names = os.listdir(audio_directory)
print(new_names)

['Tat_question_4.wav', 'Tat_question_2.mp3', 'Tat_question_1.mp3', 'Mig_question_2.mp3', 'Tat_mul_Questions.wav', 'Tat_question_3.wav', 'Mig_question_1.mp3', 'Mig_question_2.wav', 'Mig_question_3.mp3', 'Tat_question_4.mp3', 'girl_1.wav', 'Tat_mul_Questions.mp3', 'Mig_question_3.wav', 'Tat_question_1.wav', 'Tat_question_2.wav', '__MACOSX', 'Mig_question_1.wav', 'girl_1.mp3', 'Mig_question_4.wav', 'Tat_question_3.mp3', 'not_mig_1.wav', 'not_mig_1.mp3', 'Mig_question_4.mp3']


In [39]:
!pip install SpeechRecognition


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 32.1 MB/s eta 0:00:00


In [45]:
import torch
import torchaudio
import torch.nn as nn
import torchaudio.transforms as transforms
import os
import speech_recognition as sr

# CNN model with dynamic size handling for gender prediction
class GenderCNN(nn.Module):
    def __init__(self):
        super(GenderCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output for binary classification (Male/Female)

    def convs(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(x.size(0), -1)  # Flatten before passing to fully connected layers
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Load pre-trained model or train your own on a gender-labeled dataset
model = GenderCNN()

# Function to load and process audio files
def process_audio(file_path):
    try:
        waveform, sample_rate = torchaudio.load(file_path)
        mel_spectrogram = transforms.MelSpectrogram()(waveform)

        # Reshape the tensor to match the CNN input format [batch_size, channels, height, width]
        input_tensor = mel_spectrogram.unsqueeze(0)  # Add batch dimension
        input_tensor = input_tensor.mean(dim=1, keepdim=True)  # Convert to single channel if needed

        # Adjust the size dynamically
        conv_output = model.convs(input_tensor)
        flattened_size = conv_output.numel() // conv_output.size(0)
        model.fc1 = nn.Linear(flattened_size, 128)  # Adjust fully connected layer input size

        output = model(input_tensor)
        gender = torch.argmax(output, dim=1).item()
        return "Male" if gender == 0 else "Female"
    except Exception as e:
        return f"Error processing file: {e}"

# Function for speech recognition
def recognize_speech(file_path):
    try:
        recognizer = sr.Recognizer()
        with sr.AudioFile(file_path) as source:
            audio = recognizer.record(source)
            return recognizer.recognize_google(audio)
    except Exception as e:
        return f"Speech Recognition Error: {e}"

# Process each audio file in the extracted directory
audio_directory = 'audios'
for root, dirs, files in os.walk(audio_directory):
    for file_name in files:
        # Skip any files that start with '._' which are system files or files without valid audio extensions
        if file_name.startswith("._") or not file_name.lower().endswith(('.wav', '.mp3')):
            continue
        file_path = os.path.join(root, file_name)
        print(f"Processing {file_name}:")
        print("Predicted Gender:", process_audio(file_path))
        print("Recognized Speech:", recognize_speech(file_path))


Processing Tat_question_2.mp3:
Predicted Gender: Female
Recognized Speech: Speech Recognition Error: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format
Processing Tat_question_1.mp3:
Predicted Gender: Female
Recognized Speech: Speech Recognition Error: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format
Processing Mig_question_2.mp3:
Predicted Gender: Male
Recognized Speech: Speech Recognition Error: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format
Processing Mig_question_1.mp3:
Predicted Gender: Male
Recognized Speech: Speech Recognition Error: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format
Processing Mig_question_3.mp3:
Predicted Gender: Female
Recognized Speech: Speech Recognition Error: Audio file could not be rea

In [47]:
!pip install pydub


In [51]:
import torch
import torchaudio
import torch.nn as nn
import torchaudio.transforms as transforms
import zipfile
import os
import speech_recognition as sr
from pydub import AudioSegment

# CNN model with dynamic size handling for gender prediction
class GenderCNN(nn.Module):
    def __init__(self):
        super(GenderCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output for binary classification (Male/Female)

    def convs(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(x.size(0), -1)  # Flatten before passing to fully connected layers
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Load pre-trained model or train your own on a gender-labeled dataset
model = GenderCNN()

# Function to load and process audio files
def process_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    mel_spectrogram = transforms.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=512,          # Increase to capture more frequency detail
    hop_length=256,     # Adjust according to n_fft
    n_mels=40           # Lower number of Mel bands
    )(waveform)

    # Reshape the tensor to match the CNN input format [batch_size, channels, height, width]
    input_tensor = mel_spectrogram.unsqueeze(0)  # Add batch dimension
    input_tensor = input_tensor.mean(dim=1, keepdim=True)  # Convert to single channel if needed

    # Adjust the size dynamically
    conv_output = model.convs(input_tensor)
    flattened_size = conv_output.numel() // conv_output.size(0)
    model.fc1 = nn.Linear(flattened_size, 128)  # Adjust fully connected layer input size

    output = model(input_tensor)
    gender = torch.argmax(output, dim=1).item()
    return "Male" if gender == 0 else "Female"

# Function for speech recognition after converting to WAV
def recognize_speech(file_path):
    # Convert MP3 to WAV
    sound = AudioSegment.from_mp3(file_path)
    wav_path = file_path.replace(".mp3", ".wav")
    sound.export(wav_path, format="wav")

    # Perform speech recognition
    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_path) as source:
        audio = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio)
            return text
        except sr.UnknownValueError:
            return "Speech Recognition could not understand the audio"
        except sr.RequestError:
            return "Could not request results from Speech Recognition service"

# Process each audio file in the extracted directory
audio_directory = 'audios'
for root, dirs, files in os.walk(audio_directory):
    for file_name in files:
        if file_name.startswith("._") or not file_name.lower().endswith(('.wav', '.mp3')):
            continue
        file_path = os.path.join(root, file_name)
        print(f"Processing {file_name}:")
        print("Predicted Gender:", process_audio(file_path))
        print(f"Recognized Speech: {recognize_speech(file_path)} \n")


Processing Tat_question_4.wav:
Predicted Gender: Male
Recognized Speech: what is colombia's capital 

Processing Tat_question_2.mp3:
Predicted Gender: Male
Recognized Speech: how much is 2 + 2 

Processing Tat_question_1.mp3:
Predicted Gender: Female
Recognized Speech: when is Obama's birthday 

Processing Mig_question_2.mp3:
Predicted Gender: Male
Recognized Speech: how much is 2 + 2 

Processing Tat_mul_Questions.wav:
Predicted Gender: Female
Recognized Speech: how are you how are you how are you when is Obama's birthday how much is 2 + 2 what is the secret of happiness 

Processing Tat_question_3.wav:
Predicted Gender: Female
Recognized Speech: what is the secret of happiness 

Processing Mig_question_1.mp3:
Predicted Gender: Female
Recognized Speech: when is Obama's birthday 

Processing Mig_question_2.wav:
Predicted Gender: Female
Recognized Speech: how much is 2 + 2 

Processing Mig_question_3.mp3:
Predicted Gender: Male
Recognized Speech: what's the secret of happiness 

Process

In [52]:
file_names = os.listdir(audio_directory)
print(file_names)

['Tat_question_4.wav', 'Tat_question_2.mp3', 'Tat_question_1.mp3', 'Mig_question_2.mp3', 'Tat_mul_Questions.wav', 'Tat_question_3.wav', 'Mig_question_1.mp3', 'Mig_question_2.wav', 'Mig_question_3.mp3', 'Tat_question_4.mp3', 'girl_1.wav', 'Tat_mul_Questions.mp3', 'Mig_question_3.wav', 'Tat_question_1.wav', 'Tat_question_2.wav', '__MACOSX', 'Mig_question_1.wav', 'girl_1.mp3', 'Mig_question_4.wav', 'Tat_question_3.mp3', 'not_mig_1.wav', 'not_mig_1.mp3', 'Mig_question_4.mp3']


In [53]:
import zipfile
import os
# Upload and extract the zip file
from google.colab import files
uploaded = files.upload()

zip_file = 'audios.zip'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('audios')

# Process each audio file in the extracted directory
unzipped_files = 'audios'

Saving audios.zip to audios (2).zip


In [59]:
import torch
import torchaudio
import torch.nn as nn
import torchaudio.transforms as transforms
import zipfile
import os
import speech_recognition as sr
from pydub import AudioSegment

# CNN model with dynamic size handling for gender prediction
class GenderCNN(nn.Module):
    def __init__(self):
        super(GenderCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output for binary classification (Male/Female)

    def convs(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(x.size(0), -1)  # Flatten before passing to fully connected layers
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Load pre-trained model or train your own on a gender-labeled dataset
model = GenderCNN()

# Function to load and process audio files
def process_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    mel_spectrogram = transforms.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=512,          # Increase to capture more frequency detail
    hop_length=256,     # Adjust according to n_fft
    n_mels=40           # Lower number of Mel bands
    )(waveform)

    # Reshape the tensor to match the CNN input format [batch_size, channels, height, width]
    input_tensor = mel_spectrogram.unsqueeze(0)  # Add batch dimension
    input_tensor = input_tensor.mean(dim=1, keepdim=True)  # Convert to single channel if needed

    # Adjust the size dynamically
    conv_output = model.convs(input_tensor)
    flattened_size = conv_output.numel() // conv_output.size(0)
    model.fc1 = nn.Linear(flattened_size, 128)  # Adjust fully connected layer input size

    output = model(input_tensor)
    gender = torch.argmax(output, dim=1).item()
    return "Male" if gender == 0 else "Female"

# Function for speech recognition after converting to WAV
def recognize_speech(file_path):
    # Convert MP3 to WAV
    sound = AudioSegment.from_mp3(file_path)
    wav_path = file_path.replace(".mp3", ".wav")
    sound.export(wav_path, format="wav")

    # Perform speech recognition
    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_path) as source:
        audio = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio)
            return text
        except sr.UnknownValueError:
            return "Speech Recognition could not understand the audio"
        except sr.RequestError:
            return "Could not request results from Speech Recognition service"
processed_files = set()

# Process each audio file in the extracted directory
for root, dirs, files in os.walk(audio_directory):
    # Sort the files alphabetically
    files.sort()

    for file_name in files:
        if file_name.startswith("._") or not file_name.lower().endswith(('.wav', '.mp3')):
            continue

        # Get the base file name without extension
        base_name = os.path.splitext(file_name)[0]

        # Skip if already processed as another format
        if base_name in processed_files:
            continue

        file_path = os.path.join(root, file_name)
        print(f"Processing {file_name}:")
        print("Predicted Gender:", process_audio(file_path))
        print(f"Recognized Speech: {recognize_speech(file_path)} \n")

        # Mark the file as processed
        processed_files.add(base_name)


Processing Mig_question_1.mp3:
Predicted Gender: Male
Recognized Speech: when is Obama's birthday 

Processing Mig_question_2.mp3:
Predicted Gender: Male
Recognized Speech: how much is 2 + 2 

Processing Mig_question_3.mp3:
Predicted Gender: Male
Recognized Speech: what's the secret of happiness 

Processing Mig_question_4.mp3:
Predicted Gender: Female
Recognized Speech: what's colombia's capital 

Processing Tat_mul_Questions.mp3:
Predicted Gender: Male
Recognized Speech: Speech Recognition could not understand the audio 

Processing Tat_question_1.mp3:
Predicted Gender: Female
Recognized Speech: when is Obama's birthday 

Processing Tat_question_2.mp3:
Predicted Gender: Female
Recognized Speech: how much is 2 + 2 

Processing Tat_question_3.mp3:
Predicted Gender: Male
Recognized Speech: what is the secret of happiness 

Processing Tat_question_4.mp3:
Predicted Gender: Female
Recognized Speech: what is colombia's capital 

Processing girl_1.mp3:
Predicted Gender: Male
Recognized Speec

In [66]:
file_names = os.listdir(audio_directory)
for file_name in file_names:
    if file_name.endswith('.mp3') or file_name.startswith("__"):
        file_names.remove(file_name)
print(sorted(file_names))

['Mig_question_1.wav', 'Mig_question_2.wav', 'Mig_question_3.wav', 'Mig_question_4.mp3', 'Mig_question_4.wav', 'Tat_mul_Questions.wav', 'Tat_question_1.mp3', 'Tat_question_1.wav', 'Tat_question_2.wav', 'Tat_question_3.wav', 'Tat_question_4.mp3', 'Tat_question_4.wav', 'girl_1.wav', 'not_mig_1.wav']
